# Within-Project experiment

In [ ]:
import os

import matplotlib
import sklearn
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier
import math
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import SelectFromModel, chi2, SelectKBest

# Data Preprocess

In [ ]:
text_neg = []
text_pos = []


def loaddata(path):
    numwords = []
    file = open(path, encoding="utf-8")
    texts_projects = []
    label_projects = []
    for line in file.readlines():
        line = line.strip("\n")
        a = line[13:].lower()
        if a == '':
            continue
        if a == '   ':
            continue
        if a == ' ':
            continue
        if a == '  ':
            continue
        texts_projects.append(a)

        counter = len(a.split(" "))
        numwords.append(counter)
        if line[4:12] == "negative":
            label_projects.append(0)
            text_neg.append(a)
        else:
            label_projects.append(1)
            text_pos.append(a)
    return texts_projects, label_projects, numwords

# Identification

In [ ]:
path = './data'
files = os.listdir(path)
index = 0
f0= 0
for file in files:
    vocabulary = {"this": 0, "the": 1, "not": 2}
    texts, labels, numwords = loaddata(r'./data' + file)
    idf = TfidfVectorizer(lowercase=False, max_features=1000,stop_words='english')
    y = pd.Series(labels)
    y = y.values.ravel()
    x_tr, x_te, y_train, y_test = train_test_split(texts, y, test_size=0.1, shuffle=True, random_state=1)

    x_train = idf.fit_transform(x_tr)
    x_test = idf.transform(x_te)
    from lightgbm import LGBMClassifier
    from xgboost import XGBClassifier
    smo = SMOTE()
    x_train, y_train = smo.fit_sample(x_train, y_train)
    gbm0 = LGBMClassifier()
    gbm0.fit(x_train, y_train)
    y_predict = gbm0.predict(x_test)
    y_pred = gbm0.predict(x_test)
    y_predprob = gbm0.predict_proba(x_test)[:, 1]
    # print("Accuracy:%.4f" % metrics.accuracy_score(y_test, y_pred))
    tn, fp, fn, tp = metrics.confusion_matrix(y_test, y_predict).ravel()
    rec = tp / (tp + fn)
    Recall = tp / (tp + fn)
    specificity = tn / (fp + tn)
    precision = tp / (tp + fp)
    gmeans = math.sqrt(specificity * rec)
    f1 = 2 * precision * rec / (precision + rec)
    print(f1)
    f0 += f1
print(f0/10)